In [3]:
# プレトレーニング層の再利用
# 転移学習：１から大規模DNNをやるのではなく、似たようなタスクのNNを探し下位層を再利用する
# 前提として転移元と先の入力が同じような低レベル特徴量を持つ場合に可能。画像サイズなどが違う場合は前処理で調整する
# 現時点であまり活用しないと思うので実処理はgithubで見ること。

# お決まり設定
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [4]:
# import_meta_graph（）:モデルの状態を復元する関数
# グラフの操作をデフォルトのグラフにロードし、復元するために使用できるSaverを返します。
# デフォルトでは、Saverはグラフの構造を.metaファイルに保存するので、それがロードするファイルです。

In [5]:
import tensorflow as tf
reset_graph()

In [6]:
saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

In [7]:
# グラフの構造がわからない場合は、すべての操作を一覧表示
# 以下コマンドでグラフ構造が一覧出力される
# 何が何だかわからない可能性があるのでtensorboardで見るのがいい
for op in tf.get_default_graph().get_operations():
    print(op.name)

X
y
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
dnn/hidden1/Relu
hidden2/kernel/Initializer/random_uniform/shape
hidden2/kernel/Initializer/random_uniform/min
hidden2/kernel/Initializer/random_uniform/max
hidden2/kernel/Initializer/random_uniform/RandomUniform
hidden2/kernel/Initializer/random_uniform/sub
hidden2/kernel/Initializer/random_uniform/mul
hidden2/kernel/Initializer/random_uniform
hidden2/kernel
hidden2/kernel/Assign
hidden2/kernel/read
hidden2/bias/Initializer/zeros
hidden2/bias
hidden2/bias/Assign
hidden2/bias/read
dn

In [8]:
# 事前にtensorboard.py を持ってくること
from tensorboard import show_graph

In [10]:
show_graph(tf.get_default_graph())

In [ ]:
# 必要な操作がわかったら以下のメソッドで取り出せる
# get_operation_by_name（）メソッドまたはget_tensor_by_name（）メソッド

In [15]:
# 取り出し方の例
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

# accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")

training_op = tf.get_default_graph().get_operation_by_name("GradientDescent")

In [17]:
# 利用したいオペレーション全てでコレクションを作成することもできる
# そうするとSaverを利用して、モデル状態の復元と再訓練が可能
for op in (X, y, training_op): # テキストのgithubと異なりaccuracyを保存していない
    tf.add_to_collection("my_important_ops", op)